In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
import timeit
start_time = pd.datetime.now()

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [2]:
df = pd.read_csv('applications data.csv')

In [3]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [4]:
df.date = pd.to_datetime(df.date,format='%Y%m%d')

In [5]:
df.dtypes

record                  int64
date           datetime64[ns]
ssn                     int64
firstname              object
lastname               object
address                object
zip5                    int64
dob                     int64
homephone               int64
fraud_label             int64
dtype: object

### Clean frivolous fields

In [6]:
df.loc[df.ssn == 999999999, 'ssn'] = df.loc[df.ssn == 999999999, 'record']
df.loc[df.address == '123 MAIN ST', 'address'] = df.loc[df.address == '123 MAIN ST', 'record']
df.loc[df.homephone == 9999999999, 'homephone'] = df.loc[df.homephone == 9999999999, 'record']
df.loc[df.dob == 19070626, 'dob'] = df.loc[df.dob == 19070626, 'record']

In [7]:
df.ssn = df.ssn.astype(str)
df.zip5 = df.zip5.astype(str)
df.dob = df.dob.astype(str)
df.homephone = df.homephone.astype(str)
df.address = df.address.astype(str)

In [8]:
# add leading 0 to zips

df['zip5'] = df.zip5.apply(lambda x: x if len(x) == 5 else '0'*(5-len(x)) + x)

### Risk table for day of week

In [9]:
df['dow'] = df.date.apply(lambda x: calendar.day_name[x.weekday()])

In [10]:
train_test = df[df.date < '2016-11-01']

In [11]:
# do statistical smoothing
c = 4; nmid = 20; y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/c))
df['dow_risk'] = df.dow.map(y_dow_smooth)

In [12]:
df.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label,dow,dow_risk
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,02765,1,1797504115,0,Friday,0.014499
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1,Friday,0.014499
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,3,216537580,0,Friday,0.014499
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0,Friday,0.014499
4,5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,03173,19980315,6101082272,0,Friday,0.014499


### Create entities

In [13]:
df['name'] = df.firstname + df.lastname
df['fulladdress'] = df.address + df.zip5
df['name_dob'] = df.name + df.dob
df['name_fulladdress'] = df.name + df.fulladdress
df['name_homephone'] = df.name + df.homephone
df['fulladdress_dob'] = df.fulladdress + df.dob
df['fulladdress_homephone'] = df.fulladdress + df.homephone
df['dob_homephone'] = df.dob + df.homephone
df['homephone_name_dob'] = df.homephone + df.name_dob

In [14]:
for field in list(df.iloc[:,np.r_[3:9, 12:15]].columns):
    df['ssn_' + field] = df.ssn + df[field]

In [15]:
attributes = list(df.iloc[:, np.r_[2, 5, 7, 8, 12:30]].columns)

In [16]:
attributes

['ssn',
 'address',
 'dob',
 'homephone',
 'name',
 'fulladdress',
 'name_dob',
 'name_fulladdress',
 'name_homephone',
 'fulladdress_dob',
 'fulladdress_homephone',
 'dob_homephone',
 'homephone_name_dob',
 'ssn_firstname',
 'ssn_lastname',
 'ssn_address',
 'ssn_zip5',
 'ssn_dob',
 'ssn_homephone',
 'ssn_name',
 'ssn_fulladdress',
 'ssn_name_dob']

### Velocity + Day since

In [17]:
df.date = pd.to_datetime(df.date)

In [18]:
df1 = df.copy()
final = df.copy()
df1['check_date'] = df1.date
df1['check_record'] = df1.record

In [ ]:
start = timeit.default_timer()

for entity in attributes:
    
    try: print('Run time for the last entity ----------------- {}s'.format(timeit.default_timer() - st))
    except: print('\n')
    st = timeit.default_timer()
    
    df_l = df1[['record', 'date', entity]]
    df_r = df1[['check_record', 'check_date', entity]]
    
    temp = pd.merge(df_l, df_r, left_on = entity, right_on = entity)
    
    
    temp1 = temp[temp.record > temp.check_record][['record','date','check_date']]\
                                                .groupby('record')[['date', 'check_date']].last()
    mapper = (temp1.date - temp1.check_date).dt.days
    final[entity + '_day_since'] = final.record.map(mapper)
    final[entity + '_day_since'].fillna((final.date - pd.to_datetime('2016-01-01')).dt.days, inplace = True)

    print('\n' + entity + '_day_since ---> Done')
    
    for time in [0,1,3,7,14,30]:
        
        temp_1 = temp[(temp.check_date >= (temp.date - dt.timedelta(time))) &\
                       (temp.record >= temp.check_record)]
        
        col_name = entity + '_count_' + str(time)    
        mapper2 = temp_1.groupby('record')[entity].count()      
        final[col_name] = final.record.map(mapper2)
        
        print(entity + '_count_' + str(time) + ' ---> Done')

print('Total run time: {}mins'.format((timeit.default_timer() - start)/60))

### Relative Velocity

In [20]:
start = timeit.default_timer()

for att in attributes:
    for d in ['0', '1']:
        for dd in ['3', '7', '14', '30']:
            final[att + '_count_' + d + '_by_' + dd] =\
            final[att + '_count_' + d]/(final[att + '_count_' + dd]/float(dd))

print('Total run time: {}s'.format(timeit.default_timer() - start))

Total run time: 7.736342798999999s


### Keep desired variables

In [21]:
final.set_index('record', inplace = True)

In [22]:
final = final.iloc[:, np.r_[8, 10, 29:337]]
final.shape

(1000000, 310)

In [23]:
# final['fraud_label'] = df['fraud_label']

In [24]:
final.shape

(1000000, 310)

In [25]:
final.head()

,fraud_label,dow_risk,ssn_day_since,ssn_count_0,ssn_count_1,ssn_count_3,ssn_count_7,ssn_count_14,ssn_count_30,address_day_since,...,ssn_homephone_count_0_by_3,ssn_homephone_count_0_by_7,ssn_homephone_count_0_by_14,ssn_homephone_count_0_by_30,ssn_homephone_count_1_by_3,ssn_homephone_count_1_by_7,ssn_homephone_count_1_by_14,ssn_homephone_count_1_by_30,ssn_name_count_0_by_3,ssn_name_count_0_by_7
record,,,,,,,,,,,,,,,,,,,,,
1,0,0.014499,0.0,1,1,1,1,1,1,0.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
2,1,0.014499,0.0,1,1,1,1,1,1,0.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
3,0,0.014499,0.0,1,1,1,1,1,1,0.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
4,0,0.014499,0.0,1,1,1,1,1,1,0.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
5,0,0.014499,0.0,1,1,1,1,1,1,0.0,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0


In [26]:
final.to_csv('vars 308.csv')

In [27]:
print('Duration: ', pd.datetime.now() - start_time)

Duration:  0:06:55.963941
